In [1]:
# using Revise
using TaylorModels, IntervalArithmetic, TaylorSeries

In [2]:
const interval=Interval

IntervalArithmetic.Interval

In [3]:
displayBigO(false)

false

In [4]:
const Interval = IntervalArithmetic.Interval

import Base:^

^(x::Interval{Float64}, n::Int) = pow(x, n)

^ (generic function with 96 methods)

In [5]:
n = 12  # order
t, a, b = set_variables("t a b", order=n)

3-element Array{TaylorSeries.TaylorN{Float64},1}:
  1.0 t
  1.0 a
  1.0 b

In [6]:
using Plots
gr()

Plots.GRBackend()

Lotka-Volterra:

In [7]:
f1(u, v) = 2 * u * (1 - v)
f2(u, v) = -v * (1 - u)

fs = [f1, f2]   # use FunctionWrapper?

2-element Array{Function,1}:
 f1
 f2

In [8]:
h = 0.05  # time_step

bounds = IntervalBox(-h..h, -h..h)  # bounds on t, a, b

u0 = (1..1) + a   # initial conditions as function of a, b
v0 = (3..3) + b 

#@time U, V, uu_new, vv_new = Taylor_step(fs, n, u0, v0, 0..time_step, bounds);

In [9]:
u0, typeof(u0)

( [1, 1] + [1, 1] a, TaylorSeries.TaylorN{IntervalArithmetic.Interval{Float64}})

In [10]:
∫_dt(u, u0) = integrate(u, 1, u0)  

∫_dt (generic function with 1 method)

In [11]:
u0

 [1, 1] + [1, 1] a

In [12]:
integrate(u0, 1)

 [1, 1] t + [1, 1] t a

In [12]:
using TaylorModels: integral_bound

In [19]:
function Taylor_step(fs, n, u0, v0, t_interval, bounds)

    u = u0
    v = v0
    u_new = u0   # so exist outside loop
    v_new = v0

    # build up Taylor series by Picard:
    for i in 1:n+1   # how many iterations are required?
        u_new = ∫_dt(fs[1](u, v), u0)
        v_new = ∫_dt(fs[2](u, v), v0)

        u, v = u_new, v_new
    end

    # prepare Taylor Model:
    uu = TaylorNModel(n, 
                        IntervalBox(Interval(t_interval.lo), interval.(mid(bounds))...),
                        IntervalBox(t_interval, bounds...),
                        u, 
                        0..0)
    
    vv = TaylorNModel(n, 
                        IntervalBox(Interval(t_interval.lo), interval.(mid(bounds))...),
                        IntervalBox(t_interval, bounds...),
                        v, 
                        0..0)
    
    uΔ = integral_bound(fs[1](uu, vv), 1)
    vΔ = integral_bound(fs[2](uu, vv), 1)

    # make sure the intervals contain 0:
    m = mag(uΔ)
    uΔ = -m..m

    m = mag(vΔ)
    vΔ = -m..m

    @show uΔ, vΔ

    uu = TaylorNModel(n, 
                        IntervalBox(Interval(t_interval.lo), interval.(mid(bounds))...),
                        IntervalBox(t_interval, bounds...),
                        u, 
                        uΔ)
    
    vv = TaylorNModel(n, 
                        IntervalBox(Interval(t_interval.lo), interval.(mid(bounds))...),
                        IntervalBox(t_interval, bounds...),
                        v, 
                        vΔ)
    

    integ_u_bound = uΔ
    integ_v_bound = vΔ

    while ! ((integ_u_bound ⊆ uu.Δ) && (integ_v_bound ⊆ vv.Δ))
        uΔ *= 2
        vΔ *= 2

        @show uΔ, vΔ

        uu = TaylorNModel(n, 
                        IntervalBox(Interval(t_interval.lo), interval.(mid(bounds))...),
                        IntervalBox(t_interval, bounds...),
                        u, 
                        uΔ)
    
        vv = TaylorNModel(n, 
                        IntervalBox(Interval(t_interval.lo), interval.(mid(bounds))...),
                        IntervalBox(t_interval, bounds...),
                        v, 
                        vΔ)

        integ_u_bound = integral_bound(fs[1](uu, vv), 1)
        integ_v_bound = integral_bound(fs[2](uu, vv), 1)

    end


    # only need to bound the integral, not actually carry out
    # the whole integral

    # contract:

    uu_new = ∫_dt(fs[1](uu, vv), u0)
    vv_new = ∫_dt(fs[2](uu, vv), v0)

    for i in 1:10
        uu, vv = uu_new, vv_new
        uu_new = ∫_dt(fs[1](uu, vv), u0)
        vv_new = ∫_dt(fs[2](uu, vv), v0)
    end

    @show uu_new.Δ, vv_new.Δ
    
    variables = get_variables()

    # evaluate at end of time step:
    U = uu_new.p([t_interval.hi, variables[2:end]...])   # replace with partial evaluation
    V = vv_new.p([t_interval.hi, variables[2:end]...])

    return U, V, uu_new, vv_new
end

Taylor_step (generic function with 1 method)

Integration:

In [20]:
@time U, V, uu_new, vv_new = Taylor_step(fs, n, u0, v0, 0..h, bounds);

(uΔ, vΔ) = ([-1.24547e-10, 1.24547e-10], [-6.25807e-11, 6.25807e-11])
(uu_new.Δ, vv_new.Δ) = ([-9.94876e-11, 8.67548e-11], [-4.37391e-11, 5.00512e-11])
  0.919438 seconds (2.46 M allocations: 211.039 MiB, 5.21% gc time)


In [21]:
U

 [0.819119, 0.81912] + [0.813385, 0.813386] a + [-0.0815976, -0.0815975] b + [-0.00579507, -0.00579506] a² + [-0.0827449, -0.0827448] a b + [0.00412112, 0.00412113] b² + [-6.14407e-05, -6.14406e-05] a³ + [-0.0011481, -0.00114809] a² b + [0.0043226, 0.00432261] a b² + [-0.000142084, -0.000142083] b³ + [-1.87301e-07, -1.873e-07] a⁴ + [-5.83226e-07, -5.83225e-07] a³ b + [0.00020551, 0.000205511] a² b² + [-0.00015721, -0.000157209] a b³ + [3.82255e-06, 3.82256e-06] b⁴ + [-3.17709e-09, -3.17708e-09] a⁵ + [2.62969e-07, 2.6297e-07] a⁴ b + [4.09003e-06, 4.09004e-06] a³ b² + [-1.58338e-05, -1.58337e-05] a² b³ + [4.57627e-06, 4.57628e-06] a b⁴ + [-8.63939e-08, -8.63938e-08] b⁵ + [-1.30209e-10, -1.30208e-10] a⁶ + [2.30468e-08, 2.30469e-08] a⁵ b + [-1.34115e-07, -1.34114e-07] a⁴ b² + [-4.63889e-07, -4.63888e-07] a³ b³ + [8.52083e-07, 8.52084e-07] a² b⁴ + [-1.35417e-07, -1.35416e-07] a b⁵ + [1.38888e-09, 1.38889e-09] b⁶

In [25]:
bounds = IntervalBox(bounds...)

[-0.0500001, 0.0500001] × [-0.0500001, 0.0500001]

In [27]:
step = 0.1

bounds = IntervalBox(bounds...)

#h = 0.05
#bounds = (-h..h) * ones(2)  # bounds on a and b

U = (1..1) + a   # initial condition as function of a, b
V = (3..3) + b   # initial condition as function of a, b


Us = [U] 
Vs = [V] 

for i in 1:60
    @show i
    
    U, V = Taylor_step(fs, n, U, V, (i*step)..((i+1)*step), bounds);
    
    push!(Us, U)
    push!(Vs, V)
end


i = 1
(uΔ, vΔ) = ([-0.000139701, 0.000139701], [-6.98512e-05, 6.98512e-05])
(uu_new.Δ, vv_new.Δ) = ([-0.000134936, 0.000100097], [-5.00491e-05, 6.74683e-05])
i = 2
(uΔ, vΔ) = ([-0.000706519, 0.000706519], [-0.00035326, 0.00035326])
(uu_new.Δ, vv_new.Δ) = ([-0.000538767, 0.000727446], [-0.000363723, 0.000269384])
i = 3
(uΔ, vΔ) = ([-0.000326577, 0.000326577], [-0.000163289, 0.000163289])
(uu_new.Δ, vv_new.Δ) = ([-0.000307446, 0.000233469], [-0.000116735, 0.000153723])
i = 4
(uΔ, vΔ) = ([-4.05154e-05, 4.05154e-05], [-2.02577e-05, 2.02577e-05])
(uu_new.Δ, vv_new.Δ) = ([-4.56113e-05, 3.96375e-05], [-1.98188e-05, 2.28057e-05])
i = 5
(uΔ, vΔ) = ([-7.34011e-06, 7.34011e-06], [-3.67006e-06, 3.67006e-06])
(uu_new.Δ, vv_new.Δ) = ([-4.74723e-06, 6.9448e-06], [-3.4724e-06, 2.37362e-06])
i = 6
(uΔ, vΔ) = ([-1.11435e-06, 1.11435e-06], [-5.57172e-07, 5.57172e-07])
(uu_new.Δ, vv_new.Δ) = ([-7.25042e-07, 1.09389e-06], [-5.46943e-07, 3.62521e-07])
i = 7
(uΔ, vΔ) = ([-8.19405e-06, 8.19405e-06], [-4.09703

(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN

(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN

(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN

(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN

(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN

(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN

(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN, NaN])
(uΔ, vΔ) = ([NaN, NaN], [NaN

LoadError: [91mInterruptException:[39m

In [19]:
using Interact

INFO: Recompiling stale cache file /Users/dpsanders/.julia/lib/v0.6/Interact.ji for module Interact.


In [29]:
typeof(Us[1])

TaylorSeries.TaylorN{IntervalArithmetic.Interval{Float64}}

In [21]:
# Calculate sets for each step:

xsets = []
ysets = []

for i in 1:61
    print(i, " ")
    xs, ys = calculate_set(Us[i], Vs[i], bounds)
    push!(xsets, xs)
    push!(ysets, ys)
end

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 

Plot them all:

In [22]:
p = plot(xlim=(0, 4.5), ylim=(0, 3.2), leg=false, aspect_ratio=1)

for i in 1:length(xsets)
    
    plot!(xsets[i], ysets[i], linetype=:shape, alpha=0.5)

end

p

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
<polygon clip-path="url(#clip3802)" points="
153.45,40.2487 154.049,40.2487 154.648,40.2487 155.246,40.2487 155.845,40.2487 156.444,40.2487 157.042,40.2487 157.641,40.2487 158.24,40.2487 158.839,40.2487 
 159.437,40.2487 160.036,40.2487 160.635,40.2487 161.233,40.2487 161.832,40.2487 162.431,40.2487 163.029,40.2487 163.628,40.2487 164.227,40.2487 164.825,40.2487 
 164.825,40.2487 164.825,39.65 164.825,39.0513 164.825,38.4526 164.825,37.854 164.825,37.2553 164.825,36.6566 164.825,36.0579 164.825,35.4592 164.825,34.8605 
 164.825,34.2618 164.825,33.6631 164.825,33.0645 164.825,32.4658 164.825,31.8671 164.825,31.2684 164.825,30.6697 164.825,30.071 164.825,29.4723 164.825,28.8736 
 164.825,28.8736 164.227,28.8736 163.628,28.8736 163.029,28.8736 162.431,28.8736 161.832,28.8736 161.233,28.8736 160.635,28.8736 160.036,28.8736 159.437,28.8736 
 158.839,28.8736 158.24,28.8736 157.641,28.8736 157.042,28.8736 156.444,28.8736 155.845,28.8736 155.246,28.8736 154.648,28.8736 154.049,28.8736 153.45,28.8736 
 153.45,28.8736 153.45,29.4723 153.45,30.071 153.45,30.6697 153.45,31.2684 153.45,31.8671 153.45,32.4658 153.45,33.0645 153.45,33.6631 153.45,34.2618 
 153.45,34.8605 153.45,35.4592 153.45,36.0579 153.45,36.6566 153.45,37.2553 153.45,37.854 153.45,38.4526 153.45,39.0513 153.45,39.65 153.45,40.2487 
 153.45,40.2487 
 " fill="#009af9" fill-opacity="0.5"/>
<polyline clip-path="url(#clip3802)" style="stroke:#000000; stroke-width:1; stroke-opacity:0.5; fill:none" points="
 153.45,40.2487 154.049,40.2487 154.648,40.2487 155.246,40.2487 155.845,40.2487 156.444,40.2487 157.042,40.2487 157.641,40.2487 158.24,40.2487 158.839,40.2487 
 159.437,40.2487 160.036,40.2487 160.635,40.2487 161.233,40.2487 161.832,40.2487 162.431,40.2487 163.029,40.2487 163.628,40.2487 164.227,40.2487 164.825,40.2487 
 164.825,39.65 164.825,39.0513 164.825,38.4526 164.825,37.854 164.825,37.2553 164.825,36.6566 164.825,36.0579 164.825,35.4592 164.825,34.8605 
 164.825,34.2618 164.825,33.6631 164.825,33.0645 164.825,32.4658 164.825,31.8671 164.825,31.2684 164.825,30.6697 164.825,30.071 164.825,29.4723 164.825,28.8736 
 164.227,28.8736 163.628,28.8736 163.029,28.8736 162.431,28.8736 161.832,28.8736 161.233,28.8736 160.635,28.8736 160.036,28.8736 159.437,28.8736 
 158.839,28.8736 158.24,28.8736 157.641,28.8736 157.042,28.8736 156.444,28.8736 155.845,28.8736 155.246,28.8736 154.648,28.8736 154.049,28.8736 153.45,28.8736 
 153.45,29.4723 153.45,30.071 153.45,30.6697 153.45,31.2684 153.45,31.8671 153.45,32.4658 153.45,33.0645 153.45,33.6631 153.45,34.2618 
 153.45,34.8605 153.45,35.4592 153.45,36.0579 153.45,36.6566 153.45,37.2553 153.45,37.854 153.45,38.4526 153.45,39.0513 153.45,39.65 153.45,40.2487 
 
 "/>
<polygon clip-path="url(#clip3802)" points="
118.9,47.3013 119.297,47.159 119.695,47.0167 120.092,46.8743 120.489,46.7319 120.885,46.5894 121.282,46.4469 121.679,46.3043 122.075,46.1617 122.472,46.019 
 122.868,45.8763 123.264,45.7336 123.66,45.5907 124.056,45.4479 124.452,45.305 124.848,45.162 125.244,45.019 125.639,44.8759 126.034,44.7328 126.43,44.5896 
 126.43,44.5896 126.346,44.0128 126.261,43.436 126.177,42.8593 126.093,42.2826 126.01,41.7059 125.926,41.1293 125.842,40.5528 125.758,39.9762 125.675,39.3997 
 125.592,38.8233 125.508,38.2468 125.425,37.6705 125.342,37.0941 125.259,36.5178 125.176,35.9415 125.093,35.3653 125.01,34.7891 124.927,34.213 124.845,33.6369 
 124.845,33.6369 124.457,33.7834 124.07,33.9298 123.683,34.0762 123.295,34.2226 122.907,34.3689 122.519,34.5151 122.131,34.6614 121.743,34.8075 121.355,34.9536 
 120.967,35.0997 120.578,35.2457 120.19,35.3916 119.801,35.5375 119.413,35.6834 119.024,35.8292 118.635,35.9749 118.246,36.1206 117.857,36.2663 117.467,36.4119 
 117.467,36.4119 117.542,36.9847 117.617,37.5575 117.692,38.1304 117.767,38.7033 117.842,39.2763 117.917,39.8

In [45]:
@manipulate for i in slider(1:length(xsets), value=1)
    p = plot(xlim=(0, 5), ylim=(0, 3), leg=false, aspect_ratio=1)
    for j in 1:i
        plot!(xsets[j], ysets[j], linetype=:shape, alpha=0.5)
    end
    p
end


Interact.Slider{Int64}(49: "input-15" = 1 Int64 , "", 1, 1:61, "horizontal", true, "d", true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
<polygon clip-path="url(#clip5002)" points="
128.353,31.9988 128.581,31.9988 128.809,31.9988 129.037,31.9988 129.264,31.9988 129.492,31.9988 129.72,31.9988 129.948,31.9988 130.175,31.9988 130.403,31.9988 
 130.631,31.9988 130.859,31.9988 131.086,31.9988 131.314,31.9988 131.542,31.9988 131.77,31.9988 131.997,31.9988 132.225,31.9988 132.453,31.9988 132.681,31.9988 
 132.908,31.9988 133.136,31.9988 133.364,31.9988 133.592,31.9988 133.819,31.9988 134.047,31.9988 134.275,31.9988 134.502,31.9988 134.73,31.9988 134.958,31.9988 
 135.186,31.9988 135.413,31.9988 135.641,31.9988 135.869,31.9988 136.097,31.9988 136.324,31.9988 136.552,31.9988 136.78,31.9988 137.008,31.9988 137.235,31.9988 
 137.463,31.9988 137.691,31.9988 137.919,31.9988 138.146,31.9988 138.374,31.9988 138.602,31.9988 138.83,31.9988 139.057,31.9988 139.285,31.9988 139.513,31.9988 
 139.513,31.9988 139.513,31.7711 139.513,31.5433 139.513,31.3156 139.513,31.0878 139.513,30.8601 139.513,30.6323 139.513,30.4046 139.513,30.1769 139.513,29.9491 
 139.513,29.7214 139.513,29.4936 139.513,29.2659 139.513,29.0381 139.513,28.8104 139.513,28.5826 139.513,28.3549 139.513,28.1271 139.513,27.8994 139.513,27.6716 
 139.513,27.4439 139.513,27.2162 139.513,26.9884 139.513,26.7607 139.513,26.5329 139.513,26.3052 139.513,26.0774 139.513,25.8497 139.513,25.6219 139.513,25.3942 
 139.513,25.1664 139.513,24.9387 139.513,24.711 139.513,24.4832 139.513,24.2555 139.513,24.0277 139.513,23.8 139.513,23.5722 139.513,23.3445 139.513,23.1167 
 139.513,22.889 139.513,22.6612 139.513,22.4335 139.513,22.2057 139.513,21.978 139.513,21.7503 139.513,21.5225 139.513,21.2948 139.513,21.067 139.513,20.8393 
 139.513,20.8393 139.285,20.8393 139.057,20.8393 138.83,20.8393 138.602,20.8393 138.374,20.8393 138.146,20.8393 137.919,20.8393 137.691,20.8393 137.463,20.8393 
 137.235,20.8393 137.008,20.8393 136.78,20.8393 136.552,20.8393 136.324,20.8393 136.097,20.8393 135.869,20.8393 135.641,20.8393 135.413,20.8393 135.186,20.8393 
 134.958,20.8393 134.73,20.8393 134.502,20.8393 134.275,20.8393 134.047,20.8393 133.819,20.8393 133.592,20.8393 133.364,20.8393 133.136,20.8393 132.908,20.8393 
 132.681,20.8393 132.453,20.8393 132.225,20.8393 131.997,20.8393 131.77,20.8393 131.542,20.8393 131.314,20.8393 131.086,20.8393 130.859,20.8393 130.631,20.8393 
 130.403,20.8393 130.175,20.8393 129.948,20.8393 129.72,20.8393 129.492,20.8393 129.264,20.8393 129.037,20.8393 128.809,20.8393 128.581,20.8393 128.353,20.8393 
 128.353,20.8393 128.353,21.067 128.353,21.2948 128.353,21.5225 128.353,21.7503 128.353,21.978 128.353,22.2057 128.353,22.4335 128.353,22.6612 128.353,22.889 
 128.353,23.1167 128.353,23.3445 128.353,23.5722 128.353,23.8 128.353,24.0277 128.353,24.2555 128.353,24.4832 128.353,24.711 128.353,24.9387 128.353,25.1664 
 128.353,25.3942 128.353,25.6219 128.353,25.8497 128.353,26.0774 128.353,26.3052 128.353,26.5329 128.353,26.7607 128.353,26.9884 128.353,27.2162 128.353,27.4439 
 128.353,27.6716 128.353,27.8994 128.353,28.1271 128.353,28.3549 128.353,28.5826 128.353,28.8104 128.353,29.0381 128.353,29.2659 128.353,29.4936 128.353,29.7214 
 128.353,29.9491 128.353,30.1769 128.353,30.4046 128.353,30.6323 128.353,30.8601 128.353,31.0878 128.353,31.3156 128.353,31.5433 128.353,31.7711 128.353,31.9988 
 128.353,31.9988 
 " fill="#009af9" fill-opacity="0.5"/>
<polyline clip-path="url(#clip5002)" style="stroke:#000000; stroke-width:1; stroke-opacity:0.5; fill:none" points="
 128.353,31.9988 128.581,31.9988 128.809,31.9988 129.037,31.9988 129.264,31.9988 129.492,31.9988 129.72,31.9988 129.948,31.9988 130.175,31.9988 130.403,31.9988 
 130.631,31.9988 130.859,31.9988 131.086,31.9988 131.314,31.9988 131.542,31.9988 131.77,31.9988 131.997,31.9988 132.225,31.9988 132.453,31.9988 132.681,31.9988 
 132.908,31.9988 133.136,31.9988 133.364,31.99